# Tokenizer Dev

In [1]:
%load_ext autoreload
%autoreload 2

In [34]:
import os, sys
sys.path.insert(0, f"{os.getcwd()}/src")

In [127]:
import torch
from pprint import pprint

In [123]:
sentence = "Машинное обучение - это здорово! Ты молодец."
sentence_0 = "Машинное обучение - это здорово!"
sentence_1 = "Ты молодец."

# Baseline

In [85]:
#checkpoint_file='model1.pt:model2.pt:model3.pt:model4.pt'
checkpoint_file='model1.pt'
ru2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.ru-en', checkpoint_file=checkpoint_file, tokenizer='moses', bpe='fastbpe')

Using cache found in /home/stas/.cache/torch/hub/pytorch_fairseq_master


In [110]:
tokens = ru2en.tokenize(sentence)
bpe = ru2en.apply_bpe(tokens)
bpe_str = str(bpe).split()
bin = ru2en.binarize(bpe)
len(tokens), tokens
len(bpe_str), bpe_str
len(bin), bin
expected = ru2en.encode(sentence)
expected

(46, 'Машинное обучение - это здорово ! Ты молодец .')

(12, ['Ма@@', 'шин@@', 'ное', 'обучение', '-', 'это', 'здорово', '!', 'Ты', 'моло@@', 'дец', '.'])

(13, tensor([  648, 13440,    97,  3618,    25,    74, 22548,   384, 10994,  7178,
        21731,     5,     2]))

tensor([  648, 13440,    97,  3618,    25,    74, 22548,   384, 10994,  7178,
        21731,     5,     2])

# Target

In [111]:
from transformers.tokenization_fs_translator import FairseqTranslatorTokenizer

In [112]:
tokenizer = FairseqTranslatorTokenizer.from_pretrained('fairseq-translator-wmt19-ru-en')

Special tokens have been added in the vocabulary, make sure the associated word emebedding are fine-tuned or trained.


In [118]:
tokens = tokenizer.tokenize(sentence)
input_ids = tokenizer(sentence)['input_ids']
len(tokens), tokens
len(input_ids), input_ids
#tokenizer._convert_id_to_token(31232)

(12, ['Ма', 'шин', 'ное</w>', 'обучение</w>', '-</w>', 'это</w>', 'здорово</w>', '!</w>', 'Ты</w>', 'моло', 'дец</w>', '.</w>'])

(13, [648, 13440, 97, 3618, 25, 74, 22548, 384, 10994, 7178, 21731, 5, 2])

In [121]:
from numpy.testing import assert_array_equal
assert_array_equal(expected.numpy(), input_ids, "did we get it right?")

# 2 sequences

In [130]:
enc = tokenizer.encode_plus(sentence_0, sentence_1, add_special_tokens=True)
pprint(enc)
for k in enc.keys(): print(f"{k}: {len(enc[k])}")


{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'input_ids': [648,
               13440,
               97,
               3618,
               25,
               74,
               22548,
               384,
               2,
               10994,
               7178,
               21731,
               5,
               2],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]}
input_ids: 14
token_type_ids: 14
attention_mask: 14


In [131]:
"Success!!!"

'Success!!!'